In [4]:
# import the necessary packages
import time
import cv2
import cv2.aruco as aruco
import numpy as np
from realsense_depth import *
from control_class import *
from utlities import *
import sys

WIDTH = 1280
HEIGHT = 720


def drone_pose(frame ,matrix_coefficients, distortion_coefficients,corner):
    ## MARKER SIZE OF DRONE
    MARKER_SIZE = 0.056
    rvec, tvec, markerPoints = cv2.aruco.estimatePoseSingleMarkers(corner, MARKER_SIZE, matrix_coefficients,distortion_coefficients)
    return frame,tvec,rvec

def find_waypoints(frame,matrix_coefficients,distortion_coefficients,arucoDict,arucoParams):
    waypoints = np.zeros((4,3),dtype="float32")
    print(waypoints)
    found_count = 0
    MARKER_SIZE = 0.1535
    
    num = 0
    while(num<1):
        found_id = np.zeros(4)
        found_count =0
        while (found_count < 4) :
            #print('yes')
            (corners, ids, rejected) = cv2.aruco.detectMarkers(frame,arucoDict, parameters=arucoParams)
            if len(corners) >0:
                for i in range(0,len(ids)):
                    if (ids[i] ==1) :
                        continue
                    if (ids[i] ==0):
                        MARKER_SIZE = 0.056
                    else:
                        MARKER_SIZE = 0.1535
                    if ( abs(corners[i][0][1][0] - corners[i][0][0][0]) > 100 or abs(corners[i][0][1][1] - corners[i][0][0][1]) > 100 ) :
                        print('ignoring large blocks')
                        continue

                    rvec, tvec, markerPoints = cv2.aruco.estimatePoseSingleMarkers(corners[i], MARKER_SIZE, matrix_coefficients,distortion_coefficients)
                    frame,cX,cY = aruco_display(frame, corners[i]) 
                    print(ids[i],cX,cY)
                    if (ids[i] == 0 and found_id[0] == 0):
                        waypoints[3][:2] = tvec[0][0][:2]
                        waypoints[3][2] = desired_depth
                        found_id[0] =1
                        found_count +=1

                    if  (ids[i] == 2 and found_id[1] == 0):
                        waypoints[0][:2] = tvec[0][0][:2]
                        waypoints[0][2] = desired_depth
                        found_id[1] = 1
                        found_count +=1

                    if (ids[i] == 3 and found_id[2] == 0):
                        waypoints[1][:2] = tvec[0][0][:2]
                        waypoints[1][2] = desired_depth
                        found_id[2] =1
                        found_count +=1

                    if(ids[i] == 4 and found_id[3] == 0):
                        waypoints[2][:2] = tvec[0][0][:2]
                        waypoints[2][2] = desired_depth
                        found_id[3] = 1
                        found_count +=1

                
        num +=1

    #waypoints = np.multiply(waypoints,1/num)

    print('Waypoints are as follows')
    print(waypoints)
    return waypoints,frame,True

mean_roll =  1519
mean_pitch = 1492
mean_throttle = 1470

def waypoint_nav(desired_pos,eps,pitch,roll,throttle):

    num_not_detected = 0
    prev_time = time.time()
    try:
        while True:

            ret, color_frame = dc.get_frame()

            gray = cv2.cvtColor(color_frame,cv2.COLOR_BGR2GRAY)
            (corners, ids, rejected) = cv2.aruco.detectMarkers(gray,arucoDict, parameters=arucoParams)

            if len(corners)>0 and 0 in ids : 
                    #print('Tracking drone')
                    for i in range(0,len(ids)):
                        
                        if ( abs(corners[i][0][1][0] - corners[i][0][0][0]) > 100 or abs(corners[i][0][1][1] - corners[i][0][0][1]) > 100 ) :
                            print(' [ERROR] : Large Marker detected ...')
                            continue
                        if ids[i] == 0:
                            color_frame,xc,yc = aruco_display(color_frame,corners[i])
                            frame,tvec,rvec = drone_pose(color_frame,k,d,corners[i])
                            x = tvec[0][0][0]
                            y = tvec[0][0][1]

                        num_not_detected = 0
            else :
                print('[INFO] : Drone not detected ...')
                if num_not_detected > 10:
                    print('[INFO] : Drone out of range...')
                    print('[INFO] : Landing ...')
                    command.land()
                    print('[INFO] : Quitting ...')
                    break

                num_not_detected +=1
                command.boxarm()
                time.sleep(0.1)
                continue
            
            ## HOVERING 

            depth = dc.get_depth(xc,yc)
            curr_pos = np.array([x,y,depth])
        
            if(np.abs(desired_pos[0] - curr_pos[0]) < eps and np.abs(desired_pos[1] - curr_pos[1]) <eps and np.abs(desired_pos[2] - curr_pos[2]) <eps):
                print('[INFO] : Reached Waypoint')
                return True
                
            print('desired pos : ',desired_pos)
            print('curr_pos :', curr_pos )



            command.pitch = pitch
            command.roll = roll 
            command.throttle = throttle

            command.send()

            print('Pitch : ',command.pitch)
            print('Roll : ',command.roll)
            print('Throttle : ',command.throttle)

            cv2.imshow("Frame", color_frame)

            
            key = cv2.waitKey(1) & 0xFF
            # if the `q` key was pressed, break from the loop
            if key == ord("q"):
                break

            ## Updating the current time for FPS Calculation
            curr_time = time.time()
            fps = 1/(curr_time-prev_time)
            prev_time = curr_time
            print('[INFO] : fps = ',fps)

            # Restricting FPS
            # time.sleep(0.01)

    except KeyboardInterrupt:
        print('[INFO] : Keyboard Interrupt')
        command.boxarm()
        command.land()
        return False
    # except:
    #     print('[INFO] : Exception')
    #     command.boxarm()
    #     command.land()
    #     return False

#arucoDict = aruco.Dictionary_get(aruco.DICT_4X4_50)

# # defining an empty custom dictionary 
arucoDict = cv2.aruco.custom_dictionary(0, 4, 1)
# adding empty bytesList array to fill with 3 markers 
arucoDict.bytesList = np.empty(shape = (5, 2, 4), dtype = np.uint8)

# adding new markers
mybits = np.array([[0,1,0,0],[1,1,0,0],[1,0,1,0],[1,1,0,1]], dtype = np.uint8)
arucoDict.bytesList[0] = cv2.aruco.Dictionary_getByteListFromBits(mybits)
mybits = np.array([[1,1,1,1],[1,0,0,1],[1,0,0,1],[0,0,0,1],], dtype = np.uint8)
arucoDict.bytesList[1] = cv2.aruco.Dictionary_getByteListFromBits(mybits)
mybits = np.array([[0,0,0,1],[0,0,0,1],[1,0,1,0],[0,1,1,1]], dtype = np.uint8)
arucoDict.bytesList[2] = cv2.aruco.Dictionary_getByteListFromBits(mybits)
mybits = np.array([[0,0,0,0],[1,1,1,0],[1,0,1,1],[0,1,1,1],], dtype = np.uint8)
arucoDict.bytesList[3] = cv2.aruco.Dictionary_getByteListFromBits(mybits)
mybits = np.array([[0,0,1,0],[1,0,1,0],[0,0,0,0],[1,1,1,1]], dtype = np.uint8)
arucoDict.bytesList[4] = cv2.aruco.Dictionary_getByteListFromBits(mybits)

arucoParams = aruco.DetectorParameters_create()

# initialize the video stream and allow the camera sensor to warm up
print("[INFO] starting video stream...")

# Change src accordingly



calibration_matrix_path = "calibration_matrix.npy"
distortion_coefficients_path = "distortion_coefficients.npy"
    
k = np.load(calibration_matrix_path)
d = np.load(distortion_coefficients_path)
time.sleep(2.0)

# loop over the frames from the video stream


desired_depth = 2
prev_depth = 0

eps = 0.1



num_not_detected = 0
desired_pos = np.zeros(3,dtype="float32")
curr_pos = np.zeros(3,dtype="float32")

dc = DepthCamera(width=WIDTH, height=HEIGHT)
first_time = True
# Finding Waypoints using Aruco tags

while True:
    ret,frame = dc.get_frame()
    waypoints,frame,ret = find_waypoints(frame,k,d,arucoDict,arucoParams)

    if ret:
        break

command = Command("192.168.4.1")
for i in range(10):
    command.disarm()
    time.sleep(0.1)

command.takeoff()
print('Take off completed')

pitch = 1540
roll = 1519
throttle = 1465
for i in range(4):
    command.set_attitude(pitch,roll,throttle)
    time.sleep(0.1)
if( not waypoint_nav(waypoints[0],0.2,pitch,roll,throttle)):
    print("[INFO] : Aborting Mission")
    cv2.destroyAllWindows()
    dc.release()
    sys.exit(1)

pitch = 1490
roll = 1460
throttle = 1460
for i in range(10):
    command.set_attitude(pitch,roll,throttle)
    time.sleep(0.1)
if( not waypoint_nav(waypoints[1],0.2,pitch,roll,throttle)):
    print("[INFO] : Aborting Mission")
    cv2.destroyAllWindows()
    dc.release()
    sys.exit(1)

pitch = 1470
roll = 1519
throttle = 1460
for i in range(4):
    command.set_attitude(pitch,roll,throttle)
    time.sleep(0.1)
if( not waypoint_nav(waypoints[2],0.2,pitch,roll,throttle)):
    print("[INFO] : Aborting Mission")
    cv2.destroyAllWindows()
    dc.release()
    sys.exit(1)

pitch = 1540
roll = 1560
throttle = 1460
for i in range(4):
    command.set_attitude(pitch,roll,throttle)
    time.sleep(0.1)
if( not waypoint_nav(waypoints[3],0.2,pitch,roll,throttle)):
    print("[INFO] : Aborting Mission")
    cv2.destroyAllWindows()
    dc.release()
    sys.exit(1)
    
cv2.destroyAllWindows()
dc.release()



[INFO] starting video stream...
DepthCamera
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[INFO] : Drone detected
[0] 719 210
[INFO] : Drone detected
[2] 189 211
[INFO] : Drone detected
[3] 201 526
[INFO] : Drone detected
[4] 832 523
Waypoints are as follows
[[-1.448086   -0.4685127   2.        ]
 [-1.4167434   0.50311357  2.        ]
 [ 0.55190563  0.5083969   2.        ]
 [ 0.19842085 -0.5142808   2.        ]]
Take off completed
[INFO] : Drone detected
desired pos :  [-1.448086  -0.4685127  2.       ]
curr_pos : [ 0.20219822 -0.52566879  2.69800019]
Pitch :  1540
Roll :  1519
Throttle :  1465
[INFO] : fps =  6.462329690511664
[INFO] : Drone detected
desired pos :  [-1.448086  -0.4685127  2.       ]
curr_pos : [ 0.20675261 -0.54602055  2.61500001]
Pitch :  1540
Roll :  1519
Throttle :  1465
[INFO] : fps =  13.21281997971283
[INFO] : Drone detected
desired pos :  [-1.448086  -0.4685127  2.       ]
curr_pos : [ 0.21136994 -0.5588949   2.54400015]
Pitch :  1540
Roll :  1519
Throttle :

SystemExit: 1